# Cleaning of mental health factors data

In [10]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine,text

In [11]:
user = 'root'
password ='7003890541'
host = 'localhost'
port = '3306'
database = 'mental_health_final_project'

engine = create_engine(f"mysql+pymysql://{user}:{password}@{host}:{port}/{database}")

In [12]:
def sql(query ,engine = engine):
    return pd.read_sql(query,engine)

In [13]:
query = '''select * from backup_mental_health limit 5'''
sql(query=query)

,Timestamp,Gender,Country,Occupation,self_employed,family_history,treatment,Days_Indoors,Growing_Stress,Changes_Habits,Mental_Health_History,Mood_Swings,Coping_Struggles,Work_Interest,Social_Weakness,mental_health_interview,care_options
0,8/27/2014 11:29,Female,United States,Corporate,None,No,Yes,1-14 days,Yes,No,Yes,Medium,No,No,Yes,No,Not sure
1,8/27/2014 11:31,Female,United States,Corporate,None,Yes,Yes,1-14 days,Yes,No,Yes,Medium,No,No,Yes,No,No
2,8/27/2014 11:32,Female,United States,Corporate,None,Yes,Yes,1-14 days,Yes,No,Yes,Medium,No,No,Yes,No,Yes
3,8/27/2014 11:37,Female,United States,Corporate,No,Yes,Yes,1-14 days,Yes,No,Yes,Medium,No,No,Yes,Maybe,Yes
4,8/27/2014 11:43,Female,United States,Corporate,No,Yes,Yes,1-14 days,Yes,No,Yes,Medium,No,No,Yes,No,Yes


In [14]:
query = '''
    select column_name from information_schema.columns
    where table_name ='backup_mental_health'
    and table_schema = database()
'''
all_backup_data_columns = [col[0] for col in sql(query).values.tolist()]

In [15]:
all_backup_data_columns

['care_options',
 'Changes_Habits',
 'Coping_Struggles',
 'Country',
 'Days_Indoors',
 'family_history',
 'Gender',
 'Growing_Stress',
 'Mental_Health_History',
 'mental_health_interview',
 'Mood_Swings',
 'Occupation',
 'self_employed',
 'Social_Weakness',
 'Timestamp',
 'treatment',
 'Work_Interest']

In [16]:
query = f'''
    with duplicate as (
    select *,row_number() over(partition by {','.join(all_backup_data_columns)}) as rn from backup_mental_health )
    select * from duplicate where rn >1;
'''
sql(query=query)

,Timestamp,Gender,Country,Occupation,self_employed,family_history,treatment,Days_Indoors,Growing_Stress,Changes_Habits,Mental_Health_History,Mood_Swings,Coping_Struggles,Work_Interest,Social_Weakness,mental_health_interview,care_options,rn
0,8/28/2014 4:16,Female,Australia,Student,No,Yes,No,More than 2 months,Yes,Maybe,Maybe,High,No,Maybe,No,No,No,2
1,2/21/2015 4:32,Female,Ireland,Student,No,No,No,More than 2 months,Yes,Maybe,Maybe,High,No,Maybe,No,No,No,2
2,8/28/2014 4:02,Female,Netherlands,Student,No,No,No,More than 2 months,Yes,Maybe,Maybe,High,No,Maybe,No,No,No,2
3,8/27/2014 18:59,Female,New Zealand,Student,No,Yes,Yes,More than 2 months,Yes,Maybe,Maybe,High,No,Maybe,No,No,No,2
4,8/27/2014 12:51,Female,United Kingdom,Student,No,No,No,More than 2 months,Yes,Maybe,Maybe,High,No,Maybe,No,Maybe,No,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2308,8/27/2014 15:26,Male,United States,Corporate,No,Yes,Yes,More than 2 months,No,Yes,No,Medium,Yes,No,No,No,Yes,2
2309,8/27/2014 15:26,Male,United States,Housewife,No,Yes,Yes,More than 2 months,No,Yes,Yes,Low,Yes,No,No,No,Yes,2
2310,8/27/2014 15:26,Male,United States,Student,No,Yes,Yes,More than 2 months,Yes,Yes,Yes,High,Yes,Yes,Maybe,No,Yes,2
2311,8/27/2014 15:26,Male,United States,Others,No,Yes,Yes,More than 2 months,Yes,Yes,Yes,Low,Yes,No,No,No,Yes,2


`THERE ARE 2313 ROWS WHICH ARE SEEMS TO BE DATA ENTRY ERROR, WITH ONLY 1 EXTRA COPIES`

In [17]:
query = '''
    select
    (select count(*) from backup_mental_health) as row_count,
    (select count(*) from information_schema.columns
            where table_name = 'backup_mental_health'
            and table_schema = database()) as column_count
'''
sql(query=query)

,row_count,column_count
0,292364,17


`BEING A LARGE DATASET I WILL USE MYSQL SINCE IT IS FASTER`

In [18]:
query = '''
    create table data as 
        select distinct * from backup_mental_health;
'''

with engine.connect() as conn:
    conn.execute(text(query))

In [19]:
query = '''describe data'''
sql(query=query).iloc[:,:2]

,Field,Type
0,Timestamp,text
1,Gender,text
2,Country,text
3,Occupation,text
4,self_employed,text
5,family_history,text
6,treatment,text
7,Days_Indoors,text
8,Growing_Stress,text
9,Changes_Habits,text


`EVERY COLUMN IS IN TEXT FORMAT, NEED TO CHANGE SOME`

In [20]:
query = '''
    select column_name from information_schema.columns
    where table_name ='data'
    and table_schema = database()
'''
all_columns = [col[0] for col in sql(query).values.tolist()]

In [21]:
for col in all_columns:
    query = f'''
        select count(distinct {col}) as {col}_count from data;
    '''
    print(sql(query=query))
    print("-"*20)

   care_options_count
0                   3
--------------------
   Changes_Habits_count
0                     3
--------------------
   Coping_Struggles_count
0                       2
--------------------
   Country_count
0             35
--------------------
   Days_Indoors_count
0                   5
--------------------
   family_history_count
0                     2
--------------------
   Gender_count
0             2
--------------------
   Growing_Stress_count
0                     3
--------------------
   Mental_Health_History_count
0                            3
--------------------
   mental_health_interview_count
0                              3
--------------------
   Mood_Swings_count
0                  3
--------------------
   Occupation_count
0                 5
--------------------
   self_employed_count
0                    2
--------------------
   Social_Weakness_count
0                      3
--------------------
   Timestamp_count
0              580
------------

`NO COLUMN WITH ONLY 1 VALUE`

In [22]:
query = ''' alter table data
    drop column Timestamp;
'''
with engine.connect() as conn:
    conn.execute(text(query))

`DROPPING THE TIME COLUMN AS IT WILL NOT BE EFFECTIVE IN OUR PROBLEM STATEMENT`

In [23]:
query = '''
    select column_name from information_schema.columns
    where table_name ='data'
    and table_schema = database()
'''
all_columns = [col[0] for col in sql(query).values.tolist()]

In [24]:
all_columns

['care_options',
 'Changes_Habits',
 'Coping_Struggles',
 'Country',
 'Days_Indoors',
 'family_history',
 'Gender',
 'Growing_Stress',
 'Mental_Health_History',
 'mental_health_interview',
 'Mood_Swings',
 'Occupation',
 'self_employed',
 'Social_Weakness',
 'treatment',
 'Work_Interest']

In [25]:
lower_columns = [col.lower() for col in all_columns]
lower_columns

['care_options',
 'changes_habits',
 'coping_struggles',
 'country',
 'days_indoors',
 'family_history',
 'gender',
 'growing_stress',
 'mental_health_history',
 'mental_health_interview',
 'mood_swings',
 'occupation',
 'self_employed',
 'social_weakness',
 'treatment',
 'work_interest']

In [26]:
for col in all_columns:
    query = f'''
        update data
        set {col} = lower({col})
    '''
    with engine.begin() as con:
        con.execute(text(query))
    

In [27]:
query = '''select * from data limit 5'''
sql(query=query)

,Gender,Country,Occupation,self_employed,family_history,treatment,Days_Indoors,Growing_Stress,Changes_Habits,Mental_Health_History,Mood_Swings,Coping_Struggles,Work_Interest,Social_Weakness,mental_health_interview,care_options
0,female,united states,corporate,None,no,yes,1-14 days,yes,no,yes,medium,no,no,yes,no,not sure
1,female,united states,corporate,None,yes,yes,1-14 days,yes,no,yes,medium,no,no,yes,no,no
2,female,united states,corporate,None,yes,yes,1-14 days,yes,no,yes,medium,no,no,yes,no,yes
3,female,united states,corporate,no,yes,yes,1-14 days,yes,no,yes,medium,no,no,yes,maybe,yes
4,female,united states,corporate,no,yes,yes,1-14 days,yes,no,yes,medium,no,no,yes,no,yes


In [28]:
for col in all_columns:
    if col != col.lower():
        query = f'''
            alter table data
            rename column {col} to {col.lower()}
        '''
        with engine.connect() as conn:
            conn.execute(text(query))

`LOWER CASED THE COLUMN NAMES`

In [29]:
query = '''select * from data limit 5'''
sql(query=query)

,gender,country,occupation,self_employed,family_history,treatment,days_indoors,growing_stress,changes_habits,mental_health_history,mood_swings,coping_struggles,work_interest,social_weakness,mental_health_interview,care_options
0,female,united states,corporate,None,no,yes,1-14 days,yes,no,yes,medium,no,no,yes,no,not sure
1,female,united states,corporate,None,yes,yes,1-14 days,yes,no,yes,medium,no,no,yes,no,no
2,female,united states,corporate,None,yes,yes,1-14 days,yes,no,yes,medium,no,no,yes,no,yes
3,female,united states,corporate,no,yes,yes,1-14 days,yes,no,yes,medium,no,no,yes,maybe,yes
4,female,united states,corporate,no,yes,yes,1-14 days,yes,no,yes,medium,no,no,yes,no,yes


In [30]:
query = '''
    select distinct days_indoors from data
'''
sql(query=query)

,days_indoors
0,1-14 days
1,go out every day
2,more than 2 months
3,15-30 days
4,31-60 days


In [31]:
query = '''
    update data
    set days_indoors = 
    case 
    when days_indoors = '1-14 days' then '<15 days'
    when days_indoors = 'go out every day' then '0 days'
    when days_indoors = 'more than 2 months' then '2 month+'
    when days_indoors = '15-30 days' then '1 month'
    when days_indoors = '31-60 days' then '<2 month'
    else days_indoors
    end;
'''
with engine.connect() as con:
    con.execute(text(query))
    con.commit()

`FIXED SOME VALUES IN DAYS INDOORS COLUMN`

In [32]:
query = '''
    select distinct days_indoors from data
'''
sql(query=query)

,days_indoors
0,<15 days
1,0 days
2,2 month+
3,1 month
4,<2 month


In [33]:
query = '''
    select column_name from information_schema.columns
    where table_name ='data'
    and table_schema = database()
'''
all_columns = [col[0] for col in sql(query).values.tolist()]

In [34]:
for col in all_columns:
    query = f'''
        select count(*) as {col}_null from data
        where {col} is null
    '''
    print(sql(query=query))
    print("-"*30)

   care_options_null
0                  0
------------------------------
   changes_habits_null
0                    0
------------------------------
   coping_struggles_null
0                      0
------------------------------
   country_null
0             0
------------------------------
   days_indoors_null
0                  0
------------------------------
   family_history_null
0                    0
------------------------------
   gender_null
0            0
------------------------------
   growing_stress_null
0                    0
------------------------------
   mental_health_history_null
0                           0
------------------------------
   mental_health_interview_null
0                             0
------------------------------
   mood_swings_null
0                 0
------------------------------
   occupation_null
0                0
------------------------------
   self_employed_null
0                5193
------------------------------
   social_weaknes

In [35]:
query = '''
    update data
    set self_employed = 'unknown'
    where self_employed is null
'''
with engine.connect() as con:
    con.execute(text(query))
    con.commit()

`FILLED ALL THE NULL VALUES WITH A CONSTANCT VALUE - 'UNKNOWN'`

In [36]:
query = '''
    select count(*) as count from data
    where self_employed is null;
'''
sql(query=query)

,count
0,0


In [42]:
query = '''
    select * from data limit 5
'''
sql(query)

,gender,country,occupation,self_employed,family_history,treatment,days_indoors,growing_stress,changes_habits,mental_health_history,mood_swings,coping_struggles,work_interest,social_weakness,mental_health_interview,care_options
0,female,united states,corporate,unknown,no,yes,<15 days,yes,no,yes,medium,no,no,yes,no,not sure
1,female,united states,corporate,unknown,yes,yes,<15 days,yes,no,yes,medium,no,no,yes,no,no
2,female,united states,corporate,unknown,yes,yes,<15 days,yes,no,yes,medium,no,no,yes,no,yes
3,female,united states,corporate,no,yes,yes,<15 days,yes,no,yes,medium,no,no,yes,maybe,yes
4,female,united states,corporate,no,yes,yes,<15 days,yes,no,yes,medium,no,no,yes,no,yes


In [38]:
for col in all_columns:
    query = f'''
        select count(distinct {col}) as {col}_count from data;
    '''
    print(sql(query=query))
    print("-"*20)

   care_options_count
0                   3
--------------------
   changes_habits_count
0                     3
--------------------
   coping_struggles_count
0                       2
--------------------
   country_count
0             35
--------------------
   days_indoors_count
0                   5
--------------------
   family_history_count
0                     2
--------------------
   gender_count
0             2
--------------------
   growing_stress_count
0                     3
--------------------
   mental_health_history_count
0                            3
--------------------
   mental_health_interview_count
0                              3
--------------------
   mood_swings_count
0                  3
--------------------
   occupation_count
0                 5
--------------------
   self_employed_count
0                    3
--------------------
   social_weakness_count
0                      3
--------------------
   treatment_count
0                2
------------

In [46]:
for col in all_columns:
    query = f'''
        select distinct {col} from data
    '''
    print(sql(query))
    print("-"*30)

  care_options
0     not sure
1           no
2          yes
------------------------------
  changes_habits
0             no
1            yes
2          maybe
------------------------------
  coping_struggles
0               no
1              yes
------------------------------
                   country
0            united states
1                   poland
2                australia
3                   canada
4           united kingdom
5             south africa
6                   sweden
7              new zealand
8              netherlands
9                    india
10                 belgium
11                 ireland
12                  france
13                portugal
14                  brazil
15              costa rica
16                  russia
17                 germany
18             switzerland
19                 finland
20                  israel
21                   italy
22  bosnia and herzegovina
23               singapore
24                 nigeria
25                 c

`THERE IS NO INVALID VALUE`

In [48]:
query = '''
    select country,count(*) as `count` from data
    group by country
    order by count desc;
'''
sql(query)

,country,count
0,united states,169112
1,united kingdom,51356
2,canada,18699
3,australia,6014
4,netherlands,5891
5,ireland,5542
6,germany,4680
7,sweden,2812
8,india,2771
9,france,2340
